In [12]:
# !pip3 install pandas

In [13]:
import pandas as pd

In [25]:
df = pd.read_csv("transactions.csv")
df.head()

,transaction_id,customer_id,product_id,quantity,price,transaction_date
0,1,101,201,2,19.99,2023-01-15
1,2,102,202,1,99.99,2023-02-16
2,3,103,203,5,9.99,2023-03-17
3,4,101,204,1,49.99,2023-04-18
4,5,104,205,3,29.99,2023-05-19


In [26]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [27]:
last_year_transactions = df[df['transaction_date'] >= pd.Timestamp.now() - pd.DateOffset(years=1)]

last_year_transactions


,transaction_id,customer_id,product_id,quantity,price,transaction_date
7,8,106,208,1,199.99,2023-08-22
8,9,101,209,3,5.99,2023-09-23
9,10,103,210,2,14.99,2023-10-24
10,11,104,211,1,299.99,2023-11-25
11,12,105,212,6,12.99,2023-12-26
12,13,106,213,1,49.99,2024-01-27
13,14,101,214,4,3.99,2024-02-28
14,15,102,215,3,9.99,2024-03-29
15,16,103,216,1,79.99,2024-04-30
16,17,104,217,2,39.99,2024-05-01


In [28]:
total_sales_per_customer = last_year_transactions.groupby('customer_id').apply(lambda x: (x['quantity'] * x['price']).sum()).reset_index(name='total_sales')

total_sales_per_customer

,customer_id,total_sales
0,101,57.90
1,102,29.97
2,103,109.97
3,104,379.97
4,105,277.93
5,106,349.93


In [29]:
#Sort total sales in descending order
total_sales_per_customer = total_sales_per_customer.sort_values(by='total_sales', ascending=False)
total_sales_per_customer

,customer_id,total_sales
3,104,379.97
5,106,349.93
4,105,277.93
2,103,109.97
0,101,57.90
1,102,29.97


In [30]:
print(total_sales_per_customer)

   customer_id  total_sales
3          104       379.97
5          106       349.93
4          105       277.93
2          103       109.97
0          101        57.90
1          102        29.97


## Perform same query with PySpark

In [37]:
!pip3 install pyspark

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, expr

ModuleNotFoundError: No module named 'pyspark'

In [40]:
spark = SparkSession.builder.appName("Transaction session").getOrCreate

NameError: name 'SparkSession' is not defined

In [ ]:
df = spark.read_csv("transactions.csv", headers=True, inferSchema=True)

In [ ]:
df = df.withColumn('transaction_date', col('transaction_date').cast('date'))

In [ ]:
last_year_transactions = df.filter(col('transaction_date') >= expr("date_sub(current_date(), 365)"))

In [41]:
total_sales_per_customer = last_year_transactions.groupBy('customer_id').agg(spark_sum(col('quantity') * col('price')).alias('total_sales'))

AttributeError: 'DataFrame' object has no attribute 'groupBy'

In [ ]:
total_sales_per_customer = total_sales_per_customer.orderBy(col('total_sales').desc())

In [ ]:
total_sales_per_customer.show()